In [1]:
get_ipython().system(u'export LD_LIBRARY_PATH=/usr/local/cuda/lib64/')
import tensorflow as tf
tf.set_random_seed(42)
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))

In [2]:
!pip install matplotlib

In [3]:

# coding: utf-8

# In[1]:


get_ipython().system(u'pip install pillow')
get_ipython().system(u'pip install keras numpy')
get_ipython().system(u'export LD_LIBRARY_PATH=/usr/local/cuda/lib64/')
get_ipython().system(u'KERAS_BACKEND=tensorflow python -c "from keras import backend"')



# In[2]:


!pip install h5py

import h5py

import matplotlib.pyplot as plt

import keras
import numpy as np
np.random.seed(42)

from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.callbacks import TensorBoard
from keras import optimizers
from keras import applications
from keras.models import Model

tf.reset_default_graph()
run = 1

Using TensorFlow backend.


Using TensorFlow backend.


In [4]:



# In[3]:


img_width, img_height = 150, 150

train_data_dir = 'data/train'
validation_data_dir = 'data/validation'


# In[4]:


datagen = ImageDataGenerator(rescale=1./255)
train_datagen_augmented = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)
batch_size = 32


train_generator = train_datagen_augmented.flow_from_directory(
        train_data_dir,
        target_size=(150, 150),
        batch_size=batch_size,
        class_mode='binary')

validation_generator = datagen.flow_from_directory(
        validation_data_dir,
        target_size=(150, 150),
        batch_size=batch_size,
        class_mode='binary')




# In[ ]:


model = Sequential()

# first layer
model.add(Convolution2D(32,(3,3),input_shape=(img_width, img_height, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

# Second layer
model.add(Convolution2D(64,(3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

# Third Layer
model.add(Convolution2D(128,(3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

# Final Layer
model.add(Flatten()) # flatten to single vector
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid')) #to convert to probabilities for each class
model.summary()
model.compile(optimizer='rmsprop',loss='binary_crossentropy',
             metrics=['accuracy'])
#binary_crossentropy great for binary classification problems



# In[ ]:




Found 1800 images belonging to 2 classes.
Found 200 images belonging to 2 classes.


In [ ]:

# Train
epochs = 1
train_samples = 1800
validation_samples = 200



history = model.fit_generator(train_generator, steps_per_epoch=train_samples // batch_size, epochs=epochs,validation_data=validation_generator,
                   validation_steps=validation_samples,
                   callbacks = [TensorBoard("gs://test-ml-123/dogs-cats/logs/run_a")])



# In[ ]:



model.save_weights('models/basic_cnn_30_epochs.h5')

model.evaluate_generator(validation_generator, validation_samples)

# list all data in history
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

Epoch 1/20
56/56 [==============================] - 26s - loss: 0.7519 - acc: 0.5312 - val_loss: 0.6887 - val_acc: 0.5349
Epoch 2/20
56/56 [==============================] - 24s - loss: 0.7001 - acc: 0.5736 - val_loss: 0.6679 - val_acc: 0.5456
Epoch 3/20
56/56 [==============================] - 24s - loss: 0.6542 - acc: 0.6122 - val_loss: 0.6208 - val_acc: 0.6555
Epoch 4/20
56/56 [==============================] - 24s - loss: 0.6250 - acc: 0.6657 - val_loss: 0.6060 - val_acc: 0.6552
Epoch 5/20
56/56 [==============================] - 24s - loss: 0.5958 - acc: 0.6740 - val_loss: 0.5953 - val_acc: 0.6751
Epoch 6/20
56/56 [==============================] - 24s - loss: 0.5780 - acc: 0.7070 - val_loss: 0.5847 - val_acc: 0.6697
Epoch 7/20
56/56 [==============================] - 24s - loss: 0.5719 - acc: 0.7143 - val_loss: 0.5555 - val_acc: 0.7287
Epoch 8/20
56/56 [==============================] - 24s - loss: 0.5584 - acc: 0.7199 - val_loss: 0.5566 - val_acc: 0.7050
Epoch 9/20
56/56 [======

### Inspect Tensorboard
Seems that TensorBoard can only easily be run in DataLab Notebooks - connect to Jupyter VM with board forwarding and then run 'tensorboard --logdir logs' and connect via client browser.

In [ ]:

model.load_weights('models/basic_cnn_30_epochs.h5')

model.evaluate_generator(validation_generator, validation_samples)

# Phase 2: Convert This Algorithm to Cloud ML Engine

In [1]:
# Format for Cloud ML Engine
#.
#├── README.md
#├── data
#│   └── mnist.pkl
#├── setup.py
#└── trainer
#    ├── __init__.py
#    └── mnist_mlp.py

### Package Code

In [ ]:
%%writefile trainer/task.py

import tensorflow as tf
tf.set_random_seed(42)

import argparse
import h5py



import keras
import numpy as np
np.random.seed(42)

from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras import optimizers
from keras import applications
from keras.models import Model
from tensorflow.contrib.learn.python.learn import learn_runner
from tensorflow.python.lib.io import file_io

import pickle

tf.reset_default_graph()
tf.logging.set_verbosity(tf.logging.INFO)
# add images sync

# In[3]:



import urllib

testfile = urllib.URLopener()
testfile.retrieve("https://storage.googleapis.com/dhodun1-ml/dogs-cats/data.tar.gz", "data.tar.gz")

import tarfile
 
# tar file to extract
theTarFile = 'data.tar.gz'
 
# tar file path to extract
extractTarPath = '.'
 
# open the tar file
tfile = tarfile.open(theTarFile)
 
if tarfile.is_tarfile(theTarFile):
    # list all contents
    #print "tar file contents:"
    #print tfile.list(verbose=False)
    # extract all contents
    tfile.extractall(extractTarPath)
else:
    print theTarFile + " is not a tarfile."

img_width, img_height = 150, 150

train_data_dir = 'data/train_large'
validation_data_dir = 'data/validation_large'


# In[4]:


datagen = ImageDataGenerator(rescale=1./255)
train_datagen_augmented = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)
batch_size = 32


train_generator = train_datagen_augmented.flow_from_directory(
        train_data_dir,
        target_size=(150, 150),
        batch_size=batch_size,
        class_mode='binary')

validation_generator = datagen.flow_from_directory(
        validation_data_dir,
        target_size=(150, 150),
        batch_size=batch_size,
        class_mode='binary')





def train_model(job_dir='./tmp/example-5', **args):



    # In[ ]:


    model = Sequential()

    # first layer
    model.add(Convolution2D(32,(3,3),input_shape=(img_width, img_height, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))

    # Second layer
    model.add(Convolution2D(64,(3,3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))

    # Third Layer
    model.add(Convolution2D(128,(3,3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))

    # Final Layer
    model.add(Flatten()) # flatten to single vector
    model.add(Dense(64))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1))
    model.add(Activation('sigmoid')) #to convert to probabilities for each class
    model.summary()
    model.compile(optimizer='rmsprop',loss='binary_crossentropy',
                 metrics=['accuracy'])
    #binary_crossentropy great for binary classification problems

    epochs = 3
    train_samples = 1800
    validation_samples = 200



    history = model.fit_generator(train_generator, steps_per_epoch=train_samples // batch_size, epochs=epochs,validation_data=validation_generator,
                       validation_steps=validation_samples)



    # In[ ]:



    model.save_weights('basic_cnn_weights.h5')
    model.save('basic_cnn_model.h5')
    
    # Save model.h5 on to google storage
    with file_io.FileIO('basic_cnn_model.h5', mode='r') as input_f:
        with file_io.FileIO(job_dir + '/models/basic_cnn_model.h5', mode='w+') as output_f:
            output_f.write(input_f.read())
            print ('File success written to {}/basic_cnn_model'.format(job_dir))
            
    # Save history for later inspection 
    history_file = open(job_dir + '/models/cnn_training_history.obj','w')
    pickle.dump(history, history_file)
    
    
    
    print(model.evaluate_generator(validation_generator, validation_samples))

    # list all data in history
    print(history.history.keys())
    # summarize history for accuracy
    

    # In[ ]:




######CLOUD ML ENGINE BOILERPLATE CODE BELOW######
if __name__ == '__main__':
  parser = argparse.ArgumentParser()
  # Input Arguments
  parser.add_argument(
      '--output_dir',
      help='GCS location to write checkpoints and export models',
      required=True
  )
  parser.add_argument(
        '--job-dir',
        help='this model ignores this field, but it is required by gcloud',
        default='junk'
    )

  args = parser.parse_args()
  arguments = args.__dict__
  output_dir = arguments.pop('output_dir')

  train_model(**arguments)

In [ ]:
%%writefile setup.py

from setuptools import find_packages
from setuptools import setup

REQUIRED_PACKAGES = [
    'keras',
    'h5py',
    'pillow'
]

setup(
    name='trainer',
    version='0.1',
    install_requires=REQUIRED_PACKAGES,
    packages=find_packages(),
    include_package_data=True,
    description='Example running keras on gcloud ml-=engine.',
    author='Drew Hodun',
    author_email='dhodun@google.com'
)

### Train

In [2]:
GCS_BUCKET = 'gs://dhodun1-ml' #CHANGE THIS TO YOUR BUCKET
PROJECT = 'dhodun1' #CHANGE THIS TO YOUR PROJECT ID
REGION = 'us-central1' #OPTIONALLY CHANGE THIS

In [3]:
import os
os.environ['GCS_BUCKET'] = GCS_BUCKET
os.environ['PROJECT'] = PROJECT
os.environ['REGION'] = REGION

### Run Local

In [ ]:
%%bash
gcloud ml-engine local train \
   --module-name=trainer.task \
   --package-path=trainer \
   -- \
   --output_dir='./output' \
   --job-dir='.'

In [ ]:
%%bash
JOB_NAME=dogs_cats_$(date -u +%y%m%d_%H%M%S)

gcloud config set project $PROJECT

gcloud ml-engine jobs submit training $JOB_NAME \
  --job-dir $GCS_BUCKET/$JOB_NAME \
  --module-name trainer.task \
  --package-path ./trainer \
  --region $REGION \
  --scale-tier=BASIC_GPU \
  -- \
  --output_dir $GCS_BUCKET/$JOB_NAME/output

In [ ]:
%%bash
gcloud ml-engine jobs list

In [ ]:
%%bash
gcloud ml-engine jobs stream-logs dogs_cats_170826_031620

## VGG Plus Small Fully Connected Network on Small subset of the data

In [ ]:
model_vgg = applications.VGG16(include_top=False, weights='imagenet')

model_vgg.summary()

### Use VGG to pre-process Data before Transfer Learning and then Save

In [ ]:

img_width, img_height = 150, 150

train_data_dir = 'data/train_large'
validation_data_dir = 'data/validation_large'

transfer_train_generator = datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)

transfer_validation_generator = datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)

In [ ]:
train_samples = 21000
validation_samples = 3998

In [ ]:
bottleneck_features_train = model_vgg.predict_generator(transfer_train_generator, train_samples//batch_size)
np.save(open('models/bottleneck_features_train.npy', 'wb'), bottleneck_features_train)
bottleneck_features_train.shape

In [ ]:
bottleneck_features_validation = model_vgg.predict_generator(transfer_validation_generator, validation_samples//batch_size)
np.save(open('models/bottleneck_features_validation.npy', 'wb'), bottleneck_features_validation)
bottleneck_features_validation.shape

In [ ]:
train_data = np.load(open('models/bottleneck_features_train_large.npy', 'rb'))
train_labels = np.array([0] * (train_samples // 2) + [1] * (train_samples // 2))
# don't have # of samples divisible by 32, so have to truncate
train_labels = train_labels[0:20968]

validation_data = np.load(open('models/bottleneck_features_validation_large.npy', 'rb'))
validation_labels = np.array([0] * (validation_samples // 2) + [1] * (validation_samples // 2))
# don't have # of samples divisible by 32, so have to truncate
validation_labels = validation_labels[0:3966]

In [ ]:
model_top = Sequential()
model_top.add(Flatten(input_shape = train_data.shape[1:]))
model_top.add(Dense(256))
model_top.add(Activation('relu'))
model_top.add(Dropout(0.5))
model_top.add(Dense(1))
model_top.add(Activation('sigmoid'))
model_top.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])

model_top.summary()



### Train Model

In [ ]:
# Train
epochs = 50
train_samples = 1800
validation_samples = 200


history = model_top.fit(train_data, train_labels, batch_size=batch_size,
                        epochs=epochs, callbacks = [TensorBoard('./logs/run_{}'.format(run))],
                        verbose=1, validation_data=(validation_data,validation_labels))
run = run + 1



# In[ ]:



model_top.save_weights('models/transfer_learning_1.h5')

model_top.evaluate(validation_data, validation_labels, )

# list all data in history
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()